In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import datetime as dt

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

In [3]:
train = pd.read_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/train_test/ver_0802/train_0802_5.csv')
test = pd.read_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/train_test/ver_0802/test_0802_5.csv')

In [4]:
train.drop(['area_le', 'area_code','요일','season'], axis=1, inplace=True)
test.drop(['area_le', 'area_code','요일','season'],axis=1, inplace=True)


In [11]:
pd.set_option("display.max_columns", None)

In [13]:
train.head()

,yyyymmdd,area,sex,frequency,dd_mes,sum_dpth_fhsc,dd_mefs,ssrate,ss_dur,sum_ss_hr,min_tg,avg_pa,max_pa,min_pa,avg_pv,max_pv,min_pv,avg_rhm,min_rhm,sum_rn,sum_rn_dur,mi10_max_rn,hr1_max_rn,hr6_max_rn,n9_9_rn,avg_ta,max_ta,min_ta,avg_ws,max_ws,max_ins_ws,dd_mes_lag1,sum_dpth_fhsc_lag1,dd_mefs_lag1,ssrate_lag1,ss_dur_lag1,sum_ss_hr_lag1,min_tg_lag1,avg_pa_lag1,max_pa_lag1,min_pa_lag1,avg_pv_lag1,max_pv_lag1,min_pv_lag1,avg_rhm_lag1,min_rhm_lag1,sum_rn_lag1,sum_rn_dur_lag1,mi10_max_rn_lag1,hr1_max_rn_lag1,hr6_max_rn_lag1,n9_9_rn_lag1,avg_ta_lag1,max_ta_lag1,min_ta_lag1,avg_ws_lag1,max_ws_lag1,max_ins_ws_lag1,dd_mes_lag2,sum_dpth_fhsc_lag2,dd_mefs_lag2,ssrate_lag2,ss_dur_lag2,sum_ss_hr_lag2,min_tg_lag2,avg_pa_lag2,max_pa_lag2,min_pa_lag2,avg_pv_lag2,max_pv_lag2,min_pv_lag2,avg_rhm_lag2,min_rhm_lag2,sum_rn_lag2,sum_rn_dur_lag2,mi10_max_rn_lag2,hr1_max_rn_lag2,hr6_max_rn_lag2,n9_9_rn_lag2,avg_ta_lag2,max_ta_lag2,min_ta_lag2,avg_ws_lag2,max_ws_lag2,max_ins_ws_lag2,dd_mes_lag3,sum_dpth_fhsc_lag3,dd_mefs_lag3,ssrate_lag3,ss_dur_lag3,sum_ss_hr_lag3,min_tg_lag3,avg_pa_lag3,max_pa_lag3,min_pa_lag3,avg_pv_lag3,max_pv_lag3,min_pv_lag3,avg_rhm_lag3,min_rhm_lag3,sum_rn_lag3,sum_rn_dur_lag3,mi10_max_rn_lag3,hr1_max_rn_lag3,hr6_max_rn_lag3,n9_9_rn_lag3,avg_ta_lag3,max_ta_lag3,min_ta_lag3,avg_ws_lag3,max_ws_lag3,max_ins_ws_lag3,SO2,CO,O3,NO2,PM10,S02_lag1,CO_lag1,O3_lag1,NO2_lag1,PM10_lag1,S02_lag2,CO_lag2,O3_lag2,NO2_lag2,PM10_lag2,S02_lag3,CO_lag3,O3_lag3,NO2_lag3,PM10_lag3,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,rainfall_24_shift,snow_24_shift,temp_24_shift,temp_48_shift,humid_24_shift,humid_48_shift,max_temp_24_shift,min_temp_24_shift,wind_24_shift,wind_48_shift,pop_over50,num_back,심정지,뇌졸중_진단,심장병_진단,고혈압_진단,이상지질혈증_진단,뇌졸중_약물치료,심장병_약물치료,고혈압_약물치료,이상지질혈증_약물치료,뇌졸중_가족력,심장병_가족력,고혈압_가족력,day1,smoking,day_mean_smoke_over1,A03_max,A03_min,A03_mean,A03_range,A03_iqr,A03_max_24,A03_min_24,A03_mean_24,A03_range_24,A03_iqr_24,A03_max_48,A03_min_48,A03_mean_48,A03_range_48,A03_iqr_48,A07_max,A07_min,A07_mean,A07_range,A07_iqr,D02_max,D02_min,D02_median,D02_max_24,D02_min_24,D02_median_24,D02_range_24,D02_max_48,D02_min_48,D02_median_48,D02_range_48,weekday_sin,weekday_cos,season_sin,season_cos,avg_pa_7일평균,max_pa_7일평균,min_pa_7일평균,avg_ta_7일평균,max_ta_7일평균,min_ta_7일평균,avg_rhm_7일평균,min_rhm_7일평균,max_ws_7일평균,max_ins_ws_7일평균,NO2_7일평균,avg_pa_14일평균,max_pa_14일평균,min_pa_14일평균,avg_ta_14일평균,max_ta_14일평균,min_ta_14일평균,avg_rhm_14일평균,min_rhm_14일평균,max_ws_14일평균,max_ins_ws_14일평균,NO2_14일평균,주말,Mon,diff_ta
0,2012-01-01,강원,1,3,10.136053,2.155943,1.303480,35.856285,9.632506,3.444997,-9.869523,1007.235630,1009.325584,1004.004358,3.126835,4.304677,2.415617,63.236009,40.360801,1.319425,3.795499,2.565648,4.369274,0.605217,0.768841,-2.420576,2.045879,-6.453899,1.780413,5.154082,8.943097,0.425440,0.053928,0.067554,53.706797,9.603948,5.157309,-10.711643,1010.952481,1014.182062,1008.379417,2.770426,3.492906,1.906770,60.089753,35.394057,5.922542,5.522200,2.405613,4.657815,4.245112,1.807018,-2.751374,1.921806,-7.806485,1.372664,3.322841,5.392499,8.157597,1.659180,0.039479,68.229108,9.592791,6.542103,-9.870923,1014.689511,1016.430323,1013.155854,3.430026,4.089441,2.732705,72.183833,46.157230,3.723311,1.857098,1.381549,2.694366,2.072126,0.761383,-3.316662,2.957736,-7.432780,1.202674,3.046707,4.717232,0.600455,0.438470,0.342631,45.458230,9.590896,4.363502,-8.519633,1011.930912,1014.123420,1009.947116,4.112360,5.218578,2.733617,73.678905,35.325765,8.843681,3.142044,1.818329,5.026921,8.630879,3.855059,-1.116461,3.030391,-4.974100,1.262846,3.634993,5.607580,0.011632,1.124709,0.016572,0.024203,93.238136,0.001753,0.246584,0.001978,0.006568,22.574379,0.001927,0.258629,0.001775,0.006279,19.798797,0.003004,0.282738,0.001703,0.007072,21.683691,14.00509,11.402662,-2.769171,-2.780795,62.145093,54.361839,2.777686,-6.639387,2.430259,2.303822,14.00509,11.402662,-2.769171,-2.780795,62.145093,54.361839,2.777686,-6.639387,2.430259,2.303822,575828,0,884,1655,44

In [ ]:
test.head()

,yyyymmdd,area,sex,dd_mes,sum_dpth_fhsc,dd_mefs,ssrate,ss_dur,sum_ss_hr,min_tg,...,max_ta_14일평균,min_ta_14일평균,avg_rhm_14일평균,min_rhm_14일평균,max_ws_14일평균,max_ins_ws_14일평균,NO2_14일평균,주말,Mon,diff_ta
0,2016-01-01,강원,1,25.262803,8.088057,6.832984,41.929419,9.633102,4.029989,-8.917460,...,5.029691,-3.663738,67.268588,45.673341,3.848153,7.09858,0.023391,0,0,8.693428
1,2016-01-01,강원,2,25.262803,8.088057,6.832984,41.929419,9.633102,4.029989,-8.917460,...,5.029691,-3.663738,67.268588,45.673341,3.848153,7.09858,0.023391,0,0,8.693428
2,2016-01-01,경기,1,3.140683,0.247502,0.263841,19.758979,9.668681,1.907674,-6.721469,...,5.029691,-3.663738,67.268588,45.673341,3.848153,7.09858,0.023391,0,0,10.305385
3,2016-01-01,경기,2,3.140683,0.247502,0.263841,19.758979,9.668681,1.907674,-6.721469,...,5.029691,-3.663738,67.268588,45.673341,3.848153,7.09858,0.023391,0,0,10.305385
4,2016-01-01,경남,1,0.415067,0.216473,0.155119,86.066663,9.814577,8.444625,-6.450657,...,5.029691,-3.663738,67.268588,45.673341,3.848153,7.09858,0.023391,0,0,11.711911


In [ ]:

print(train.shape)
print(test.shape)

(49674, 229)
(12444, 228)


In [ ]:
train.isna().sum().sum()

0

In [ ]:
test.isna().sum().sum()

0

In [ ]:
train.columns[-30:]

Index(['D02_range_48', 'weekday_sin', 'weekday_cos', 'season_sin',
       'season_cos', 'avg_pa_7일평균', 'max_pa_7일평균', 'min_pa_7일평균',
       'avg_ta_7일평균', 'max_ta_7일평균', 'min_ta_7일평균', 'avg_rhm_7일평균',
       'min_rhm_7일평균', 'max_ws_7일평균', 'max_ins_ws_7일평균', 'NO2_7일평균',
       'avg_pa_14일평균', 'max_pa_14일평균', 'min_pa_14일평균', 'avg_ta_14일평균',
       'max_ta_14일평균', 'min_ta_14일평균', 'avg_rhm_14일평균', 'min_rhm_14일평균',
       'max_ws_14일평균', 'max_ins_ws_14일평균', 'NO2_14일평균', '주말', 'Mon',
       'diff_ta'],
      dtype='object')

In [ ]:
#sex,   -> one hot encoding  

#train=pd.get_dummies(train,columns=['sex'])
#test=pd.get_dummies(test,columns=['sex'])

#test

In [ ]:
train_gangwon = train[train["area"]=="강원"]
train_genggi = train[train["area"]=="경기"]
train_gengnam = train[train["area"]=="경남"]
train_gengbook = train[train["area"]=="경북"]
train_gwangju = train[train["area"]=="광주"]
train_daegu = train[train["area"]=="대구"]
train_daejun = train[train["area"]=="대전"]
train_busan = train[train["area"]=="부산"]
train_seoul = train[train["area"]=="서울"]
train_sejong= train[train["area"]=="세종"]
train_ulsan= train[train["area"]=="울산"]
train_incheon = train[train["area"]=="인천"]
train_jeonam = train[train["area"]=="전남"]
train_jeonbook = train[train["area"]=="전북"]
train_jeju = train[train["area"]=="제주"]
train_chungnam = train[train["area"]=="충남"]
train_chungbook = train[train["area"]=="충북"]


test_gangwon = test[test["area"]=="강원"]
test_genggi = test[test["area"]=="경기"]
test_gengnam = test[test["area"]=="경남"]
test_gengbook = test[test["area"]=="경북"]
test_gwangju = test[test["area"]=="광주"]
test_daegu = test[test["area"]=="대구"]
test_daejun = test[test["area"]=="대전"]
test_busan = test[test["area"]=="부산"]
test_seoul = test[test["area"]=="서울"]
test_sejong= test[test["area"]=="세종"]
test_ulsan= test[test["area"]=="울산"]
test_incheon = test[test["area"]=="인천"]
test_jeonam = test[test["area"]=="전남"]
test_jeonbook = test[test["area"]=="전북"]
test_jeju = test[test["area"]=="제주"]
test_chungnam = test[test["area"]=="충남"]
test_chungbook = test[test["area"]=="충북"]


In [ ]:
train_gangwon_y = train_gangwon['frequency']
train_gangwon_x = train_gangwon.drop(["yyyymmdd",'frequency','area'], axis=1)
train_gangwon_x['sex'] = train_gangwon_x['sex'].astype(object)
train_gangwon_x = pd.get_dummies(train_gangwon_x, columns=[ 'sex'])

train_genggi_y = train_genggi['frequency']
train_genggi_x = train_genggi.drop(["yyyymmdd",'frequency','area'], axis=1)
train_genggi_x['sex'] = train_genggi_x['sex'].astype(object)
train_genggi_x = pd.get_dummies(train_genggi_x, columns=[ 'sex'])

train_gengnam_y = train_gengnam['frequency']
train_gengnam_x = train_gengnam.drop(["yyyymmdd",'frequency','area'], axis=1)
train_gengnam_x['sex'] = train_gengnam_x['sex'].astype(object)
train_gengnam_x = pd.get_dummies(train_gengnam_x, columns=[ 'sex'])


train_gengbook_y = train_gengbook['frequency']
train_gengbook_x = train_gengbook.drop(["yyyymmdd",'frequency','area'], axis=1)
train_gengbook_x['sex'] = train_gengbook_x['sex'].astype(object)
train_gengbook_x = pd.get_dummies(train_gengbook_x, columns=[ 'sex'])


train_gwangju_y = train_gwangju['frequency']
train_gwangju_x = train_gwangju.drop(["yyyymmdd",'frequency','area'], axis=1)
train_gwangju_x['sex'] = train_gwangju_x['sex'].astype(object)
train_gwangju_x = pd.get_dummies(train_gwangju_x, columns=[ 'sex'])

train_daegu_y = train_daegu['frequency']
train_daegu_x = train_daegu.drop(["yyyymmdd",'frequency','area'], axis=1)
train_daegu_x['sex'] = train_daegu_x['sex'].astype(object)
train_daegu_x = pd.get_dummies(train_daegu_x, columns=[ 'sex'])

train_daejun_y = train_daejun['frequency']
train_daejun_x = train_daejun.drop(["yyyymmdd",'frequency','area'], axis=1)
train_daejun_x['sex'] = train_daejun_x['sex'].astype(object)
train_daejun_x = pd.get_dummies(train_daejun_x, columns=[ 'sex'])

train_busan_y = train_busan['frequency']
train_busan_x = train_busan.drop(["yyyymmdd",'frequency','area'], axis=1)
train_busan_x['sex'] = train_busan_x['sex'].astype(object)
train_busan_x = pd.get_dummies(train_busan_x, columns=[ 'sex'])

train_seoul_y = train_seoul['frequency']
train_seoul_x = train_seoul.drop(["yyyymmdd",'frequency','area'], axis=1)
train_seoul_x['sex'] = train_seoul_x['sex'].astype(object)
train_seoul_x = pd.get_dummies(train_seoul_x, columns=[ 'sex'])

train_sejong_y = train_sejong['frequency']
train_sejong_x = train_sejong.drop(["yyyymmdd",'frequency','area'], axis=1)
train_sejong_x['sex'] = train_sejong_x['sex'].astype(object)
train_sejong_x = pd.get_dummies(train_sejong_x, columns=[ 'sex'])

train_ulsan_y = train_ulsan['frequency']
train_ulsan_x = train_ulsan.drop(["yyyymmdd",'frequency','area'], axis=1)
train_ulsan_x['sex'] = train_ulsan_x['sex'].astype(object)
train_ulsan_x = pd.get_dummies(train_ulsan_x, columns=[ 'sex'])

train_incheon_y = train_incheon['frequency']
train_incheon_x = train_incheon.drop(["yyyymmdd",'frequency','area'], axis=1)
train_incheon_x['sex'] = train_incheon_x['sex'].astype(object)
train_incheon_x = pd.get_dummies(train_incheon_x, columns=[ 'sex'])

train_jeonam_y = train_jeonam['frequency']
train_jeonam_x = train_jeonam.drop(["yyyymmdd",'frequency','area'], axis=1)
train_jeonam_x['sex'] = train_jeonam_x['sex'].astype(object)
train_jeonam_x = pd.get_dummies(train_jeonam_x, columns=[ 'sex'])

train_jeonbook_y = train_jeonbook['frequency']
train_jeonbook_x = train_jeonbook.drop(["yyyymmdd",'frequency','area'], axis=1)
train_jeonbook_x['sex'] = train_jeonbook_x['sex'].astype(object)
train_jeonbook_x = pd.get_dummies(train_jeonbook_x, columns=[ 'sex'])

train_jeju_y = train_jeju['frequency']
train_jeju_x = train_jeju.drop(["yyyymmdd",'frequency','area'], axis=1)
train_jeju_x['sex'] = train_jeju_x['sex'].astype(object)
train_jeju_x = pd.get_dummies(train_jeju_x, columns=[ 'sex'])

train_chungnam_y = train_chungnam['frequency']
train_chungnam_x = train_chungnam.drop(["yyyymmdd",'frequency','area'], axis=1)
train_chungnam_x['sex'] = train_chungnam_x['sex'].astype(object)
train_chungnam_x = pd.get_dummies(train_chungnam_x, columns=[ 'sex'])

train_chungbook_y = train_chungbook['frequency']
train_chungbook_x = train_chungbook.drop(["yyyymmdd",'frequency','area'], axis=1)
train_chungbook_x['sex'] = train_chungbook_x['sex'].astype(object)
train_chungbook_x = pd.get_dummies(train_chungbook_x, columns=[ 'sex'])


In [ ]:

test_gangwon_x = test_gangwon.drop(["area","yyyymmdd"], axis=1)
test_gangwon_x['sex'] = test_gangwon_x['sex'].astype(object)
test_gangwon_x = pd.get_dummies(test_gangwon_x, columns=[ 'sex'])

test_genggi_x = test_genggi.drop(["area","yyyymmdd"], axis=1)
test_genggi_x['sex'] = test_genggi_x['sex'].astype(object)
test_genggi_x = pd.get_dummies(test_genggi_x, columns=[ 'sex'])

test_gengnam_x = test_gengnam.drop(["area","yyyymmdd"], axis=1)
test_gengnam_x['sex'] = test_gengnam_x['sex'].astype(object)
test_gengnam_x = pd.get_dummies(test_gengnam_x, columns=[ 'sex'])

test_gengbook_x = test_gengbook.drop(["area","yyyymmdd"], axis=1)
test_gengbook_x['sex'] = test_gengbook_x['sex'].astype(object)
test_gengbook_x = pd.get_dummies(test_gengbook_x, columns=[ 'sex'])


test_gwangju_x = test_gwangju.drop(["area","yyyymmdd"], axis=1)
test_gwangju_x['sex'] = test_gwangju_x['sex'].astype(object)
test_gwangju_x = pd.get_dummies(test_gwangju_x, columns=[ 'sex'])

test_daegu_x = test_daegu.drop(["area","yyyymmdd"], axis=1)
test_daegu_x['sex'] = test_daegu_x['sex'].astype(object)
test_daegu_x = pd.get_dummies(test_daegu_x, columns=[ 'sex'])

test_daejun_x = test_daejun.drop(["area","yyyymmdd"], axis=1)
test_daejun_x['sex'] = test_daejun_x['sex'].astype(object)
test_daejun_x = pd.get_dummies(test_daejun_x, columns=[ 'sex'])

test_busan_x = test_busan.drop(["area","yyyymmdd"], axis=1)
test_busan_x['sex'] = test_busan_x['sex'].astype(object)
test_busan_x = pd.get_dummies(test_busan_x, columns=[ 'sex'])

test_seoul_x = test_seoul.drop(["area","yyyymmdd"], axis=1)
test_seoul_x['sex'] = test_seoul_x['sex'].astype(object)
test_seoul_x = pd.get_dummies(test_seoul_x, columns=[ 'sex'])

test_sejong_x = test_sejong.drop(["area","yyyymmdd"], axis=1)
test_sejong_x['sex'] = test_sejong_x['sex'].astype(object)
test_sejong_x = pd.get_dummies(test_sejong_x, columns=[ 'sex'])

test_ulsan_x = test_ulsan.drop(["area","yyyymmdd"], axis=1)
test_ulsan_x['sex'] = test_ulsan_x['sex'].astype(object)
test_ulsan_x = pd.get_dummies(test_ulsan_x, columns=[ 'sex'])

test_incheon_x = test_incheon.drop(["area","yyyymmdd"], axis=1)
test_incheon_x['sex'] = test_incheon_x['sex'].astype(object)
test_incheon_x = pd.get_dummies(test_incheon_x, columns=[ 'sex'])

test_jeonam_x = test_jeonam.drop(["area","yyyymmdd"], axis=1)
test_jeonam_x['sex'] = test_jeonam_x['sex'].astype(object)
test_jeonam_x = pd.get_dummies(test_jeonam_x, columns=[ 'sex'])

test_jeonbook_x = test_jeonbook.drop(["area","yyyymmdd"], axis=1)
test_jeonbook_x['sex'] = test_jeonbook_x['sex'].astype(object)
test_jeonbook_x = pd.get_dummies(test_jeonbook_x, columns=[ 'sex'])

test_jeju_x = test_jeju.drop(["area","yyyymmdd"], axis=1)
test_jeju_x['sex'] = test_jeju_x['sex'].astype(object)
test_jeju_x = pd.get_dummies(test_jeju_x, columns=[ 'sex'])

test_chungnam_x = test_chungnam.drop(["area","yyyymmdd"], axis=1)
test_chungnam_x['sex'] = test_chungnam_x['sex'].astype(object)
test_chungnam_x = pd.get_dummies(test_chungnam_x, columns=[ 'sex'])


test_chungbook_x = test_chungbook.drop(["area","yyyymmdd"], axis=1)
test_chungbook_x['sex'] = test_chungbook_x['sex'].astype(object)
test_chungbook_x = pd.get_dummies(test_chungbook_x, columns=[ 'sex'])

In [ ]:
xgb_model_gangwon = RandomForestRegressor( random_state=22)
xgb_model_gangwon.fit(train_gangwon_x, train_gangwon_y)
xgb_pred_gangwon = xgb_model_gangwon.predict(test_gangwon_x)
result_gangwon= test_gangwon[["yyyymmdd","area","sex"]]
result_gangwon["frequency"] = xgb_pred_gangwon


xgb_model_genggi = RandomForestRegressor( random_state=22)
xgb_model_genggi.fit(train_genggi_x, train_genggi_y)
xgb_pred_genggi = xgb_model_genggi.predict(test_genggi_x)
result_genggi= test_genggi[["yyyymmdd","area","sex"]]
result_genggi["frequency"] = xgb_pred_genggi


xgb_model_gengnam = RandomForestRegressor( random_state=22)
xgb_model_gengnam.fit(train_gengnam_x, train_gengnam_y)
xgb_pred_gengnam = xgb_model_gengnam.predict(test_gengnam_x)
result_gengnam= test_gengnam[["yyyymmdd","area","sex"]]
result_gengnam["frequency"] = xgb_pred_gengnam


xgb_model_gengbook = RandomForestRegressor(random_state=22)
xgb_model_gengbook.fit(train_gengbook_x, train_gengbook_y)
xgb_pred_gengbook = xgb_model_gengbook.predict(test_gengbook_x)
result_gengbook= test_gengbook[["yyyymmdd","area","sex"]]
result_gengbook["frequency"] = xgb_pred_gengbook

xgb_model_gwangju = RandomForestRegressor( random_state=22)
xgb_model_gwangju.fit(train_gwangju_x, train_gwangju_y)
xgb_pred_gwangju = xgb_model_gwangju.predict(test_gwangju_x)
result_gwangju= test_gwangju[["yyyymmdd","area","sex"]]
result_gwangju["frequency"] = xgb_pred_gwangju


xgb_model_daegu = RandomForestRegressor( random_state=22)
xgb_model_daegu.fit(train_daegu_x, train_daegu_y)
xgb_pred_daegu = xgb_model_daegu.predict(test_daegu_x)
result_daegu= test_daegu[["yyyymmdd","area","sex"]]
result_daegu["frequency"] = xgb_pred_daegu


xgb_model_daejun = RandomForestRegressor( random_state=22)
xgb_model_daejun.fit(train_daejun_x, train_daejun_y)
xgb_pred_daejun= xgb_model_daejun.predict(test_daejun_x)
result_daejun= test_daejun[["yyyymmdd","area","sex"]]
result_daejun["frequency"] = xgb_pred_daejun


xgb_model_busan = RandomForestRegressor(random_state=22)
xgb_model_busan.fit(train_busan_x, train_busan_y)
xgb_pred_busan = xgb_model_busan.predict(test_busan_x)
result_busan= test_busan[["yyyymmdd","area","sex"]]
result_busan["frequency"] = xgb_pred_busan


xgb_model_seoul = RandomForestRegressor(random_state=22)
xgb_model_seoul.fit(train_seoul_x, train_seoul_y)
xgb_pred_seoul = xgb_model_seoul.predict(test_seoul_x)
result_seoul= test_seoul[["yyyymmdd","area","sex"]]
result_seoul["frequency"] = xgb_pred_seoul

xgb_model_sejong = RandomForestRegressor( random_state=22)
xgb_model_sejong.fit(train_sejong_x, train_sejong_y)
xgb_pred_sejong = xgb_model_sejong.predict(test_sejong_x)
result_sejong= test_sejong[["yyyymmdd","area","sex"]]
result_sejong["frequency"] = xgb_pred_sejong



xgb_model_ulsan = RandomForestRegressor( random_state=22)
xgb_model_ulsan.fit(train_ulsan_x, train_ulsan_y)
xgb_pred_ulsan= xgb_model_ulsan.predict(test_ulsan_x)
result_ulsan= test_ulsan[["yyyymmdd","area","sex"]]
result_ulsan["frequency"] = xgb_pred_ulsan


xgb_model_incheon = RandomForestRegressor( random_state=22)
xgb_model_incheon.fit(train_incheon_x, train_incheon_y)
xgb_pred_incheon = xgb_model_incheon.predict(test_incheon_x)
result_incheon= test_incheon[["yyyymmdd","area","sex"]]
result_incheon["frequency"] = xgb_pred_incheon


xgb_model_jeonam = RandomForestRegressor(random_state=22)
xgb_model_jeonam.fit(train_jeonam_x, train_jeonam_y)
xgb_pred_jeonam = xgb_model_jeonam.predict(test_jeonam_x)
result_jeonam= test_jeonam[["yyyymmdd","area","sex"]]
result_jeonam["frequency"] = xgb_pred_jeonam


xgb_model_jeonbook = RandomForestRegressor(random_state=22)
xgb_model_jeonbook.fit(train_jeonbook_x, train_jeonbook_y)
xgb_pred_jeonbook = xgb_model_jeonbook.predict(test_jeonbook_x)
result_jeonbook= test_jeonbook[["yyyymmdd","area","sex"]]
result_jeonbook["frequency"] = xgb_pred_jeonbook


xgb_model_jeju = RandomForestRegressor( random_state=22)
xgb_model_jeju.fit(train_jeju_x, train_jeju_y)
xgb_pred_jeju = xgb_model_jeju.predict(test_jeju_x)
result_jeju = test_jeju[["yyyymmdd","area","sex"]]
result_jeju["frequency"] = xgb_pred_jeju


xgb_model_chungnam = RandomForestRegressor(random_state=22)
xgb_model_chungnam.fit(train_chungnam_x, train_chungnam_y)
xgb_pred_chungnam = xgb_model_chungnam.predict(test_chungnam_x)
result_chungnam= test_chungnam[["yyyymmdd","area","sex"]]
result_chungnam["frequency"] = xgb_pred_chungnam


xgb_model_chungbook = RandomForestRegressor(random_state=22)
xgb_model_chungbook.fit(train_chungbook_x, train_chungbook_y)
xgb_pred_chungbook = xgb_model_chungbook.predict(test_chungbook_x)
result_chungbook= test_chungbook[["yyyymmdd","area","sex"]]
result_chungbook["frequency"] = xgb_pred_chungbook

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [ ]:
result= pd.concat([result_gangwon, result_genggi], axis=0)
result= pd.concat([result, result_gengnam], axis=0)
result= pd.concat([result, result_gengbook], axis=0)
result= pd.concat([result, result_gwangju], axis=0)
result= pd.concat([result, result_daegu], axis=0)
result= pd.concat([result, result_daejun], axis=0)
result= pd.concat([result, result_busan], axis=0)
result= pd.concat([result, result_seoul], axis=0)
result= pd.concat([result, result_sejong], axis=0)
result= pd.concat([result, result_ulsan], axis=0)
result= pd.concat([result, result_incheon], axis=0)
result= pd.concat([result, result_jeonam], axis=0)
result= pd.concat([result, result_jeonbook], axis=0)
result= pd.concat([result, result_jeju], axis=0)
result= pd.concat([result, result_chungnam], axis=0)
result= pd.concat([result, result_chungbook], axis=0)

result

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,3.24
1,2016-01-01,강원,2,2.91
34,2016-01-02,강원,1,0.59
35,2016-01-02,강원,2,0.61
68,2016-01-03,강원,1,0.77
...,...,...,...,...
12375,2016-12-29,충북,2,1.01
12408,2016-12-30,충북,1,1.15
12409,2016-12-30,충북,2,1.23
12442,2016-12-31,충북,1,0.91


In [ ]:
result=result.sort_values(["yyyymmdd","sex"])
result.head(40)

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,3.24
2,2016-01-01,경기,1,5.92
4,2016-01-01,경남,1,3.38
6,2016-01-01,경북,1,4.40
8,2016-01-01,광주,1,1.51
10,2016-01-01,대구,1,2.36
12,2016-01-01,대전,1,1.07
14,2016-01-01,부산,1,3.57
16,2016-01-01,서울,1,6.96
18,2016-01-01,세종,1,0.08


In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/2-2_검증데이터셋.csv', encoding = 'cp949')
sub.head(40)

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,NaN
1,2016-01-01,경기,1,NaN
2,2016-01-01,경남,1,NaN
3,2016-01-01,경북,1,NaN
4,2016-01-01,광주,1,NaN
5,2016-01-01,대구,1,NaN
6,2016-01-01,대전,1,NaN
7,2016-01-01,부산,1,NaN
8,2016-01-01,서울,1,NaN
9,2016-01-01,세종,1,NaN


In [ ]:
sub

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,NaN
1,2016-01-01,경기,1,NaN
2,2016-01-01,경남,1,NaN
3,2016-01-01,경북,1,NaN
4,2016-01-01,광주,1,NaN
...,...,...,...,...
12439,2016-12-31,전남,2,NaN
12440,2016-12-31,전북,2,NaN
12441,2016-12-31,제주,2,NaN
12442,2016-12-31,충남,2,NaN


In [ ]:
result2= pd.merge(sub,result, on=["yyyymmdd","area","sex"], how="left" )
result2

,yyyymmdd,area,sex,frequency_x,frequency_y
0,2016-01-01,강원,1,NaN,3.24
1,2016-01-01,경기,1,NaN,5.92
2,2016-01-01,경남,1,NaN,3.38
3,2016-01-01,경북,1,NaN,4.40
4,2016-01-01,광주,1,NaN,1.51
...,...,...,...,...,...
12439,2016-12-31,전남,2,NaN,1.36
12440,2016-12-31,전북,2,NaN,0.95
12441,2016-12-31,제주,2,NaN,0.51
12442,2016-12-31,충남,2,NaN,1.12


In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/2-2_검증데이터셋.csv', encoding = 'cp949')
sub['frequency'] = result2["frequency_y"]

sub

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,3.24
1,2016-01-01,경기,1,5.92
2,2016-01-01,경남,1,3.38
3,2016-01-01,경북,1,4.40
4,2016-01-01,광주,1,1.51
...,...,...,...,...
12439,2016-12-31,전남,2,1.36
12440,2016-12-31,전북,2,0.95
12441,2016-12-31,제주,2,0.51
12442,2016-12-31,충남,2,1.12


In [ ]:
for i in range(0,sub.shape[0]) :
  if sub.iloc[i,3] < 0 :
    sub.iloc[i,3] = 0

In [ ]:
sub[sub["frequency"]<0]

,yyyymmdd,area,sex,frequency


In [ ]:
sub.to_csv('/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/220119_21_2.csv', index=False)